## Almacenamiento en frío con DynamoDB

DynamoDB ofrece dos clases de almacenamiento distintas: 

* Estándar 
* Estándar de Acceso Poco Frecuente (Estándar-IA)

La clase **Estándar** es la configuración predeterminada y suele ser adecuada para la mayoría de las cargas de trabajo. Por otro lado, la clase **Estándar-IA** está diseñada específicamente para tablas que almacenan datos a los que se accede con poca frecuencia. Esta clase ofrece un precio más rentable por gigabyte (GB) de datos almacenados. A medida que los datos envejecen y se accede a ellos con menos frecuencia, migrar de una tabla Standard a una Standard-IA se convierte en una estrategia rentable. 

Las organizaciones pueden ahorrar en costes de almacenamiento a la vez que garantizan el mismo rendimiento y las mismas integraciones que la clase de tabla Estándar de DynamoDB. Este enfoque permite a las empresas alcanzar un equilibrio entre la optimización de costes y la disponibilidad de los datos, haciendo un uso eficiente de las capacidades de almacenamiento de DynamoDB. Los clientes que tengan tablas DynamoDB en las que el almacenamiento represente aproximadamente el 50% o más de sus costes deberían considerar trasladar sus datos a una tabla Standard-IA.

El siguiente diagrama ilustra la arquitectura de la solución.

![image](https://nathanagez.com/assets/images/diagram-43a73fb3442365f6ca4d01c287bb09f9.png)

El flujo de trabajo contiene los siguientes pasos:
1. DynamoDB TTL elimina los elementos caducados de las tablas estándar de DynamoDB en función de un atributo de elemento.
2. DynamoDB Streams genera registros de flujos que contienen los elementos caducados.
3. Lambda procesa el evento de eliminación de DynamoDB Streams. Con el filtrado de eventos de Lambda, Lambda solo es invocado por eventos de eliminación de DynamoDB TTL.
4. Los datos se escriben en la tabla DynamoDB Standard-IA.


In [1]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime, timedelta
import time
import uuid
import random

#### Crear una tabla Standard-IA

In [2]:
dt_ia = DynamoTable(profile_name='637423169504_AdministratorAccess')
try:
    dt_ia.select_table('IA-TableExample')
    print(dt_ia)
except:
    dt_ia.create_table(
        table_name='IA-TableExample',
        partition_key='PK',
        partition_key_type='S',
        #infrequent_access=True
    )

Table info:
 - Table name: IA-TableExample
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/IA-TableExample
 - Table creation: 2024-08-06T12:12:35
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Table class: STANDARD_INFREQUENT_ACCESS
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF



In [3]:
dt_ia.infrequent_access = True

In [4]:
dt_ia

Table info:
 - Table name: IA-TableExample
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/IA-TableExample
 - Table creation: 2024-08-06T12:12:35
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Table class: STANDARD_INFREQUENT_ACCESS
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF

### Crear una tabla standard

In [5]:
dt = DynamoTable(profile_name='637423169504_AdministratorAccess')
try:
    dt.select_table('StandardTableExample')
    print(dt)
except:
    dt.create_table(
        table_name='StandardTableExample',
        partition_key='PK',
        partition_key_type='S',
        infrequent_access=False
    )

Table info:
 - Table name: StandardTableExample
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/StandardTableExample
 - Table creation: 2024-08-06T13:06:51
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: NEW_AND_OLD_IMAGES
 - Stream view type: NEW_AND_OLD_IMAGES



In [7]:
today = datetime.now().strftime('%Y-%m-%d')
def generate_data(total_items=300):
    count_total = 0  
    UserName, SessionId, CreationTime, ExpirationTime, SessionInfo = [], [], [], [], []
    for i in range(total_items):
        if count_total == total_items:
            break
        else:
            count_total += 1
            now_time = datetime.now()
            # TTL in 7 days
            # ttl_time = now_time + timedelta(days=7)
            ttl_time = now_time
            uuid_session = str(uuid.uuid4())
            SessionId.append(uuid_session)
            CreationTime.append(int(now_time.timestamp()))
            ExpirationTime.append(int(ttl_time.timestamp()))
            SessionInfo.append({'SessionId': uuid_session, 'UserName': 'user'+str(random.randint(1, 100)), 'Date': today})
        
    df_main = pd.DataFrame(
        {
            'PK': SessionId,
            'CreationTime': CreationTime,
            'ExpirationTime': ExpirationTime,
            'SessionInfo': SessionInfo
        }
    )   
    return df_main

df = generate_data(total_items=10)
dt.batch_pandas(df)

Data loaded successfully in 2.39 seconds.


### Eliminar datos con DynamoDB TTL

DynamoDB TTL ofrece una forma cómoda de gestionar el ciclo de vida de sus datos en DynamoDB. Con TTL, puede asignar una marca de tiempo a cada elemento de su tabla, indicando cuándo se considera caducado o que ya no es necesario. Una vez transcurrido el tiempo especificado, DynamoDB elimina automáticamente el elemento de la tabla, por lo que no es necesario eliminarlo manualmente. 

La principal ventaja del TTL es que permite reducir los volúmenes de datos almacenados eliminando elementos obsoletos o irrelevantes sin sobrecarga operativa. Esto puede resultar especialmente útil en situaciones como la descrita anteriormente, en la que existen grandes cantidades de datos que se vuelven obsoletos con el tiempo. Puede mantener su tabla aligerada y asegurarse de que solo conserva los datos más relevantes y actuales para su carga de trabajo eliminando automáticamente los elementos caducados.


In [5]:
dt.table.meta.client.update_time_to_live(
    TableName=dt.table_name,
    TimeToLiveSpecification={
        'Enabled': True,
        'AttributeName': 'ExpirationTime'
    }
)

{'TimeToLiveSpecification': {'Enabled': True,
  'AttributeName': 'ExpirationTime'},
 'ResponseMetadata': {'RequestId': 'O06L5UKJ7U9NGGK67MAROKA7O7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 06 Aug 2024 16:07:24 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '77',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'O06L5UKJ7U9NGGK67MAROKA7O7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1019368986'},
  'RetryAttempts': 0}}

### DynamoDB Streams
DynamoDB Streams proporciona un registro ordenado por tiempo que contiene los cambios realizados en los elementos de una tabla de DynamoDB. Cuando una aplicación crea, actualiza o elimina un elemento de una tabla, se escribe un registro de la modificación en la secuencia correspondiente de la tabla.


In [6]:
dt.status_stream = 'ON'

DynamoDB streams turned on successfully.


### Crear una función Lambda que procese los eventos de eliminación de DynamoDB Streams
La función debe tener el siguiente filtro de eventos de DynamoDB Streams:

* `{ "eventName": ["REMOVE"]}`

Y la variable de entorno `TABLE_NAME` con el nombre de la tabla infrecuent access.

In [9]:
dt_ia.table_name

'IA-TableExample'

Codigo de la función:

In [10]:
import boto3
import json
import os

table_name = os.getenv("TABLE_NAME")
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

def lambda_handler(event, context):
    item_saved = 0
    total_item = 0
    
    for record in event['Records']:
        if record['eventName'] == 'REMOVE':
            total_item += 1
            pk = record['dynamodb']['Keys']['PK']['S']
            creation_time = record['dynamodb']['OldImage'].get('CreationTime')
            session_info = record['dynamodb']['OldImage'].get('SessionInfo')

            item = {
                'PK': {'S': pk},
            }
            
            if creation_time: item['CreationTime'] = {'N': creation_time['N']}
            if session_info: item['SessionInfo'] = {'M': session_info['M']}
                
            try:
                response = dynamodb.put_item(
                    TableName=table_name,
                    Item=item
                )
            except:
                print(f"[ERROR] Item cannot be stored in DynamoDBB: \n {record}")
            else:
                print("Item saved.", item)
                item_saved += 1
    
    if total_item != item_saved:
        error_count = len(event['Records']) - item_saved
        msg = f'{item_saved} elements stored in DynamoDB table. {error_count} errors found.'
    else:
        msg = f'{item_saved} elements stored in DynamoDB table.'
    
    print(msg)
    return {
        'statusCode': 200,
        'body': json.dumps(msg)
    }
